# CODE START

In [ ]:
import os
import re

import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
import torch
from torch import nn as nn

from google.colab import drive

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from joblib import parallel_backend
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive

/content/drive


In [ ]:
cd MyDrive/

/content/drive/MyDrive


In [ ]:
cd 'CS3244 Project '

/content/drive/.shortcut-targets-by-id/1NohDS0PqY4h4zT4jT9WEqQL2g8QZoAHq/CS3244 Project 


#Format data

In [ ]:
filename = 'weekly_data.csv'
df = pd.read_csv(filename)

In [ ]:
df.columns

In [ ]:
df

,Epidemiology Wk,Start,End,BOR,Mean Daily Rainfall Total (mm),PSI,Mean Temperature (Celsius),Temperature Standard Dev,live_births (month div 4),no_public_holidays,waiting_time_for_admissions (hr),Paratyphoid,Typhoid,Acute Viral Hepatitis A,Acute Viral Hepatitis E,Dengue,DHF,Malaria,Chikungunya,HFMD,Measles,Mumps,Rubella,Acute Viral hepatitis B,Legionellosis,Campylobacter enteritis,Acute Viral hepatitis C,Leptospirosis,Melioidosis,Meningococcal Infection,Pertussis,Pneumococcal Disease (invasive),Haemophilus influenzae type b,Salmonellosis(non-enteric fevers),Zika,Murine Typhus,Acute Upper Respiratory Tract infections,Acute Conjunctivitis,Acute Diarrhoea,Chickenpox,Fatal_Road_Accidents,Non_Fatal_Road_Accidents,Fatal_Road_Accidents_Casualties,Non_Fatal_Road_Accidents_Casualties,Total_Hospital_Discharge,COVID_covid_discharged_from hospital,COVID_staying_in_ICU,COVID_staying_in_general_ward,COVID_total_hospitalized
0,1,31/12/2017,6/1/2018,NaN,68.556000,49.000000,25.01429,0.405457,829,1.0,NaN,1.000000,1.000000,1.000000,0.000000,83.0000,0.00000,2.000000,0.000000,326.0000,1.00000,6.000000,0.000000,1.000000,0.000000,6.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,0.000000,27.000000,0.000000,0.000000,3158.000,103.000000,580.00000,24.000000,2.258000,158.74200,NaN,NaN,8790.41935,0.000000,0.000000,0.00000,0.00000
1,2,7/1/2018,13/1/2018,NaN,11.343660,41.571429,26.32526,0.844125,829,0.0,NaN,1.000000,1.000000,1.000000,1.000000,68.0000,0.00000,1.000000,0.000000,478.0000,1.00000,13.000000,0.000000,2.000000,0.000000,8.000000,0.000000,0.000000,1.000000,1.000000,3.000000,3.000000,0.000000,33.000000,0.000000,0.000000,3632.000,93.000000,604.00000,17.000000,2.258000,158.74200,NaN,NaN,8828.12903,0.000000,0.000000,0.00000,0.00000
2,3,14/1/2018,20/1/2018,NaN,21.103750,48.571429,24.63232,1.677330,829,0.0,NaN,0.000000,0.000000,1.000000,1.000000,54.0000,0.00000,0.000000,0.000000,542.0000,2.00000,11.000000,0.000000,1.000000,0.000000,4.000000,0.000000,1.000000,1.000000,0.000000,2.000000,5.000000,0.000000,28.000000,1.000000,0.000000,4234.000,96.000000,583.00000,20.000000,2.258000,158.74200,NaN,NaN,8828.12903,0.000000,0.000000,0.00000,0.00000
3,4,21/1/2018,27/1/2018,NaN,5.088398,49.857143,26.74425,0.916314,829,0.0,NaN,1.000000,3.000000,0.000000,2.000000,45.0000,0.00000,0.000000,0.000000,645.0000,0.00000,12.000000,0.000000,1.000000,2.000000,9.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,23.000000,0.000000,0.000000,4048.000,91.000000,607.00000,19.000000,2.258000,158.74200,NaN,NaN,8828.12903,0.000000,0.000000,0.00000,0.00000
4,5,28/1/2018,3/2/2018,NaN,10.938360,55.428571,27.02920,0.694710,778.875,0.0,NaN,0.000000,0.000000,1.000000,0.000000,48.0000,0.00000,0.000000,0.000000,757.0000,1.00000,17.000000,0.000000,1.000000,0.000000,9.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,26.000000,0.000000,0.000000,3960.000,109.000000,583.00000,17.000000,2.576000,155.63800,NaN,NaN,8737.85945,0.000000,0.000000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,13,28/03/2021,03/04/2021,NaN,NaN,51.000000,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.857143,0.428571,26.42857,26.85714
170,14,04/04/2021,10/04/2021,NaN,NaN,NaN,NaN,NaN,variance,NaN,NaN,0.234855,1.764229,1.493731,0.989786,134849.2614,1.55896,0.489152,1.108199,130975.4886,5.28973,10.589391,0.095238,0.669977,0.461398,16.090307,0.313962,0.776064,0.789025,0.166667,1.863976,2.408284,0.056002,92.361581,0.135954,0.198295,1134421.062,538.042278,25230.15452,30.430893,0.312233,42.00897,0.553563,1368.944313,NaN,NaN,NaN,NaN,NaN
171,15,11/04/2021,17/04/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
no_traffic = df.drop(columns = ['Fatal_Road_Accidents',
       'Non_Fatal_Road_Accidents'])
dropped_data = no_traffic.dropna()
dropped_data.to_csv(r'formatted_unnormalised.csv')

In [ ]:
dropped_data

In [ ]:
# for pytorch tensors
ROW_COUNT = 100
COL_COUNT = 100
LABELS = []
for i in range(ROW_COUNT):
    row = [0] * COL_COUNT
    row[i] = 1
    LABELS.append(row)

print(LABELS)

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Read formatted data

In [ ]:
# sliding window
# predict for next week
# classification of current week
# TODO : feed in the previous weeks BOR as well?

In [ ]:
a = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],[17,18,19,20]]
print(get_sliding_window(a,3))

5
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]


In [ ]:
#define constants
dataname = 'formatted_unnormalised.csv'
TEST_PROP = 0.2
WINDOW_SIZE = 4 # 4 weeks worth of features
possible_labels = [14,15,16,17,18,19] # for confusion matrix

data = pd.read_csv(dataname)
data = data.drop(columns = ['Unnamed: 0','Epidemiology Wk','Start','End'])
labels = data['BOR'].shift (-WINDOW_SIZE).dropna()
data = data.drop(columns = 'BOR').iloc[:-1]
print(data)

# 0-75 75-80 80-85 85-90 90-95 95-100
# Quantising labels into buckets of 5
# 1:0-5, 2:5-10, ... 15:70-75, 16:75-80, 17:80-85. 18:85-90, 19:90-95, 20:95-100
labels = labels.to_numpy()
labels = (labels*20).astype(int)
print(labels.shape)

def get_columns(data, window_size): # returns concatenated columns of windowsize and move as the last element
  output = []
  for i in range(window_size):
      output.extend(data.columns)
  return output

def get_sliding_window(data, window_size):
  output = []
  row = [] # [no of features * window_size]
  for i in range(window_size): # memoise first row of window size n
    row.extend(data[i])
  output.append(row)
  row = row[len(data[0]):]

  for j in range(len(data) - window_size):
    row.extend(data[j + window_size]) # extend list with the next frame
    output.append(row)
    row = row[len(data[0]):]
    # print("inserting ", j, "th row")
  return output


# Get sliding window for features
features = data.to_numpy()
# print(np.array(features[0]).shape)
abs_scaler = MaxAbsScaler()
abs_scaler.fit(features)
# print(abs_scaler.max_abs_)
features = abs_scaler.transform(features)
features = get_sliding_window(features.tolist(), WINDOW_SIZE)
# print(features.shape)

# df = pd.DataFrame(data = features, columns= get_columns(data, WINDOW_SIZE))
# features = df.to_numpy()

# Split training(and validation data) with test data
# x is features, y is labels
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = TEST_PROP)
# print('x train is\n', x_train)
# print('x test is\n', x_test)
# print('y train is\n', y_train)
# print('y test is\n', y_test)


data is       Mean Daily Rainfall Total (mm)  ...  COVID_total_hospitalized
0                          6.803836  ...                   0.00000
1                          4.004416  ...                   0.00000
2                         13.412830  ...                   0.00000
3                          1.234492  ...                   0.00000
4                          9.941379  ...                   0.00000
..                              ...  ...                       ...
108                       13.529140  ...                  30.28571
109                        4.404748  ...                  36.00000
110                        7.745429  ...                  27.57143
111                        6.531124  ...                  24.14286
112                       11.089850  ...                  31.71429

[113 rows x 40 columns]
shape of labels (110,)


In [ ]:
df

,Mean Daily Rainfall Total (mm),PSI,Mean Temperature (Celsius),Temperature Standard Dev,live_births (month div 4),no_public_holidays,waiting_time_for_admissions (hr),Paratyphoid,Typhoid,Acute Viral Hepatitis A,Acute Viral Hepatitis E,Dengue,DHF,Malaria,Chikungunya,HFMD,Measles,Mumps,Rubella,Acute Viral hepatitis B,Legionellosis,Campylobacter enteritis,Acute Viral hepatitis C,Leptospirosis,Melioidosis,Meningococcal Infection,Pertussis,Pneumococcal Disease (invasive),Haemophilus influenzae type b,Salmonellosis(non-enteric fevers),Zika,Murine Typhus,Acute Upper Respiratory Tract infections,Acute Conjunctivitis,Acute Diarrhoea,Chickenpox,COVID_covid_discharged_from hospital,COVID_staying_in_ICU,COVID_staying_in_general_ward,COVID_total_hospitalized,...,Mean Daily Rainfall Total (mm),PSI,Mean Temperature (Celsius),Temperature Standard Dev,live_births (month div 4),no_public_holidays,waiting_time_for_admissions (hr),Paratyphoid,Typhoid,Acute Viral Hepatitis A,Acute Viral Hepatitis E,Dengue,DHF,Malaria,Chikungunya,HFMD,Measles,Mumps,Rubella,Acute Viral hepatitis B,Legionellosis,Campylobacter enteritis,Acute Viral hepatitis C,Leptospirosis,Melioidosis,Meningococcal Infection,Pertussis,Pneumococcal Disease (invasive),Haemophilus influenzae type b,Salmonellosis(non-enteric fevers),Zika,Murine Typhus,Acute Upper Respiratory Tract infections,Acute Conjunctivitis,Acute Diarrhoea,Chickenpox,COVID_covid_discharged_from hospital,COVID_staying_in_ICU,COVID_staying_in_general_ward,COVID_total_hospitalized
0,0.246157,0.578147,0.959731,0.696751,0.927844,0.0,0.566291,0.0,0.090909,0.166667,0.00,0.041876,0.000,0.000000,0.0,1.000000,0.117647,0.529412,1.0,0.00,0.333333,0.45,0.0,0.00,0.333333,0.0,0.0,0.571429,0.0,0.319444,0.0,0.0,0.655648,0.608,0.749669,0.476190,0.000000,0.000000,0.000000,0.000000,...,0.044663,0.573997,0.950987,0.689601,0.959253,0.0,0.556832,0.0,0.000000,0.000000,0.25,0.035176,0.00,0.333333,0.0,0.830409,0.000000,0.470588,0.0,0.25,0.000000,0.60,0.0,0.50,0.333333,0.0,0.2,0.428571,0.0,0.472222,0.0,0.0,0.740500,0.696,0.802649,0.619048,0.000000,0.000000,0.000000,0.000000
1,0.144877,0.586445,0.950558,0.597305,0.959253,0.0,0.507823,0.0,0.181818,0.166667,0.75,0.041876,0.125,0.000000,0.0,0.692982,0.000000,0.235294,0.0,0.00,0.333333,0.20,0.0,0.25,0.333333,0.0,0.2,0.428571,0.0,0.527778,0.0,0.0,0.630401,0.544,0.660927,0.619048,0.000000,0.000000,0.000000,0.000000,...,0.359671,0.597510,0.929490,0.766584,0.959253,0.0,0.582742,0.5,0.090909,0.166667,0.00,0.022334,0.00,0.666667,0.0,0.909357,0.000000,0.411765,1.0,0.25,0.666667,0.55,1.0,0.00,0.000000,0.0,0.4,0.285714,0.0,0.388889,0.0,0.0,0.720198,0.632,0.756291,0.714286,0.000000,0.000000,0.000000,0.000000
2,0.485266,0.582296,0.918943,0.593058,0.959253,0.0,0.554025,0.0,0.000000,0.333333,0.50,0.027359,0.000,0.000000,0.0,0.868421,0.000000,0.588235,1.0,0.00,0.000000,0.65,0.0,0.25,0.000000,0.0,0.0,0.428571,0.0,0.416667,0.0,0.0,0.752993,0.656,0.785430,0.761905,0.000000,0.000000,0.000000,0.000000,...,0.019729,0.600277,0.990996,0.347421,0.994058,0.0,0.617784,0.0,0.000000,0.333333,0.00,0.028476,0.00,0.333333,0.0,0.856725,0.000000,0.529412,0.0,0.50,0.000000,0.40,0.0,0.00,1.000000,0.0,0.4,0.285714,1.0,0.416667,0.0,0.5,0.692087,0.664,0.765563,0.714286,0.000000,0.000000,0.000000,0.000000
3,0.044663,0.573997,0.950987,0.689601,0.959253,0.0,0.556832,0.0,0.000000,0.000000,0.25,0.035176,0.000,0.333333,0.0,0.830409,0.000000,0.470588,0.0,0.25,0.000000,0.60,0.0,0.50,0.333333,0.0,0.2,0.428571,0.0,0.472222,0.0,0.0,0.740500,0.696,0.802649,0.619048,0.000000,0.000000,0.000000,0.000000,...,0.084528,0.533887,0.986672,0.697442,0.994058,0.0,0.509022,0.0,0.181818,0.000000,0.25,0.028476,0.00,0.333333,0.0,0.799708,0.000000,0.470588,0.0,0.25,0.000000,0.40,0.0,0.25,0.333333,0.5,0.8,0.428571,0.0,0.500000,0.0,0.0,0.691307,0.608,0.764238,0.714286,0.000000,0.000000,0.000000,0.000000
4,0.359671,0.597510,0.929490,0.766584,0.959253,0.0,0.582742,0.5,0.090909,0.166667,0.00,0.022334,0.000,0.666667,0.0,0.909357,0.000000,0.411765,1.0,0.25,0.666667,0.5

In [ ]:
# Decision Tree
decision_tree = DecisionTreeClassifier(random_state=42)
dt_params = {'max_leaf_nodes': list(range(2, 1000)), 'min_samples_split': list(range(3,10))}
dt_grid_search_cv = GridSearchCV(decision_tree, dt_params, verbose=1, cv=3)

with parallel_backend('threading', n_jobs = -1):
  dt = dt_grid_search_cv.fit(x_train, y_train)

y_predicted = dt.predict(x_test)
dt_cm = confusion_matrix(y_test, y_predicted, labels = possible_labels)
dt_score = dt.score(x_test, y_test)
dt_best_model = dt.best_estimator_
# dt_cv_score = cross_val_score(dt, x_test,y_test).mean()

print("Decision tree model is\n", dt_best_model)
print("Decsiion tree score is:", dt_score)
print("Decision tree confusion matrix is\n", dt_cm)

Fitting 3 folds for each of 6986 candidates, totalling 20958 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed:   17.5s
[Parallel(

In [ ]:
gs_searchparam = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'gamma':['scale', 'auto'],'C':[1, 10, 1000, 10000]}
params = {'max_leaf_nodes': list(range(2, 1000)), 'min_samples_split': list(range(3,10))}
decision_tree = DecisionTreeClassifier(random_state=42)
dt_grid_search_cv = GridSearchCV(decision_tree, params, verbose=1, cv=3)

with parallel_backend('threading', n_jobs = -1):
  dt_grid_search_cv.fit(x_train, y_train)


# Using Pytorch

In [ ]:
# data_1 = no_traffic
# BOR_column = data_1['BOR']
# data_1 = data_1.drop(columns = ['BOR', 'Epidemiology Wk','Start','End'])

# abs_scaler = MaxAbsScaler()
# abs_scaler.fit(data_1)
# abs_scaler.max_abs_
# scaled_data = abs_scaler.transform(data_1)
# data_1 = pd.DataFrame(scaled_data, columns = data_1.columns)
# data_1['BOR'] = BOR_column.reset_index()['BOR'] # add back the move column



In [ ]:
# data_2 = no_traffic.drop(columns = ['Epidemiology Wk','Start','End'])

In [ ]:
# data_1

In [ ]:
# data.info()

In [ ]:
# idx = round(data.iloc[3]['BOR'] * 100)
# new_label = [0] * 100 # will print all 0s in array of size no_of_labels
# new_label[idx] = 1
# print(new_label)
# data_2['live_births (month div 4)'] = data_2['live_births (month div 4)'].astype('float', errors = 'raise' )
# torch.# data_2['live_births (month div 4)'] = data_2['live_births (month div 4)'].astype('float', errors = 'raise' )(get_features(data_2))

In [ ]:
def get_labels(df, no_of_labels):
  labels = []
  for i in range(df.shape[0]):
    # idx = int(df.iloc[i]['BOR'] - 1)
    idx = round(df.iloc[i]['BOR'] * 20)
    if idx == 20:
      idx = 19
    new_label = [0] * no_of_labels # will print all 0s in array of size no_of_labels
    new_label[idx] = 1 # set the to high the position
    # print(idx)
    labels.append(new_label)
  return labels

def get_features(df):
  data = []
  df = df.drop(columns=['BOR'])
  for i in range(df.shape[0]):
    data.append(df.iloc[i])
  return data

no_of_labels = 20
# Convert to tensor
data_tensor = torch.FloatTensor(get_features(data_2))
label = torch.FloatTensor(get_labels(data_2, no_of_labels))
print(data_tensor.size())
print(label.size())

test_prop = 0.2
# Split to test-train data sets
data_train, data_test, label_train, label_test = train_test_split(data_tensor, label, test_size = test_prop)

torch.Size([114, 40])
torch.Size([114, 20])


In [ ]:
data_tensor.size()

torch.Size([114, 40])

In [ ]:
data_train.size()

torch.Size([91, 40])

In [ ]:
class Network(nn.Module):
        def __init__(self, input_size, hidden_size, h2_size, h3_size, output_size):
            super(Network, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.hidden2_size = h2_size
            self.hidden3_size = h3_size
            self.output_size = output_size

            self.relu = nn.ReLU()
            self.sigmoid = nn.Sigmoid()

            self.h_layer = nn.Linear(self.input_size, self.hidden_size)
            self.h2_layer = nn.Linear(self.hidden_size, self.hidden2_size)
            self.h3_layer = nn.Linear(self.hidden2_size, self.hidden3_size)
            self.output = nn.Linear(self.hidden3_size, self.output_size)

        def forward(self, x):
            x = self.h_layer(x)
            x = self.relu(x)
            # x = self.sigmoid(x)
            x = self.h2_layer(x)
            x = self.relu(x)
            # x = self.sigmoid(x)
            x = self.h3_layer(x)
            x = self.relu(x)
            # x = self.sigmoid(x)
            x = self.output(x)
            # x = self.relu(x)
            x = self.sigmoid(x)
            return x

In [ ]:
# HYPERPARAMETERS:
hidden_size = 1000
h2_size = 1000
h3_size = 1000
learn_rate = 0.01
no_of_iter = 1000

# can change:
# optimiser function
# loss function
# activation function
# number of layers

in_size = data_tensor.size()[1]
out_size = label.size()[1]

# Create Model
model = Network(in_size, hidden_size,h2_size, h3_size ,out_size)
# Select Loss function
criterion = nn.CrossEntropyLoss()
# Optimaiser function
optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)

In [ ]:
sum = 0
times = 20
for iteration in range (times):
  print("Iteration ", iteration)
  data_train, data_test, label_train, label_test = train_test_split(data_tensor, label, test_size = test_prop)
  model = Network(in_size, hidden_size,h2_size, h3_size ,out_size)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)
  model.train()
  for epoch in range(no_of_iter):
    optimizer.zero_grad()
    y_pred = model(data_train) #forward pass
    criterion = torch.nn.CrossEntropyLoss()
    loss = criterion(y_pred, torch.max(label_train, 1)[1])
    loss.backward()
    optimizer.step()

  total = data_test.size(0)
  correct = 0
  model.eval()
  outputs = model(data_test)
  _, ans = torch.max(label_test.data,1)
  _, predicted = torch.max(outputs.data, 1)
  print('Ground Truth: ',ans)
  print("Classified:   ", predicted)

  for i in range (data_test.size(0)):
    if (ans[i] == predicted[i].item()):
      correct += 1

  print('Accuracy of the network: %d %%' % (100 * correct / total))
  sum += 100 * correct/total
print("===============================")
print("Total Average accuracy: " + str(sum/times))

# From Matt

In [ ]:
# Download the dataset (I stored mine on dropbox; Set querystring dl=1 if u're using dropbox)
!wget -O data.csv https://www.dropbox.com/s/x47z4susaog0o2b/outputfile.csv?dl=1

--2021-04-05 03:33:43--  https://www.dropbox.com/s/x47z4susaog0o2b/outputfile.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/x47z4susaog0o2b/outputfile.csv [following]
--2021-04-05 03:33:43--  https://www.dropbox.com/s/dl/x47z4susaog0o2b/outputfile.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4811166c6c74a9682683eaa2d6.dl.dropboxusercontent.com/cd/0/get/BMC_A23gGiiCPdcvdsK0phzcEsqQVBFqF28rEoBnGwOkSR2TCMa0Ch9xZLX0cpBlPTQZyN1qVbt3b-FlR304LafbYsTsCDj1MwKuon1u7NmA12fchij-mLSFRT6vEw4REScv4zS3FF4WUor80XOtzdXZ/file?dl=1# [following]
--2021-04-05 03:33:44--  https://uc4811166c6c74a9682683eaa2d6.dl.dropboxusercontent.com/cd/0/get/BMC_A23gGiiCPdcvdsK0phzcEsqQVBFqF28rEoBnGwOkSR2TCMa0Ch9xZLX0cpBlPTQZyN1qVbt3b-FlR3

In [ ]:
import pandas as pd
# Read Data into a dataframe
df = pd.read_csv('data.csv', delimiter=',', index_col=False, header=0)
print(df)

                   Station  ...  Max Wind Speed (km/h)
0               Paya Lebar  ...                   35.3
1               Paya Lebar  ...                   22.3
2               Paya Lebar  ...                   37.1
3               Paya Lebar  ...                   33.5
4               Paya Lebar  ...                   25.9
...                    ...  ...                    ...
83710  Pasir Ris (Central)  ...                    NaN
83711  Pasir Ris (Central)  ...                    NaN
83712  Pasir Ris (Central)  ...                    NaN
83713  Pasir Ris (Central)  ...                    NaN
83714  Pasir Ris (Central)  ...                    NaN

[83715 rows x 13 columns]


In [ ]:
# Convert the columns into numeric to manipulate them
df['Mean Temperature (C)'] = pd.to_numeric(df['Mean Temperature (C)'], errors='coerce')
df['Maximum Temperature (C)'] = pd.to_numeric(df['Maximum Temperature (C)'], errors='coerce')
df['Minimum Temperature (C)'] = pd.to_numeric(df['Minimum Temperature (C)'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83715 entries, 0 to 83714
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Station                        83715 non-null  object 
 1   Year                           83321 non-null  float64
 2   Month                          83321 non-null  float64
 3   Day                            83321 non-null  float64
 4   Daily Rainfall Total (mm)      78874 non-null  float64
 5   Highest 30 Min Rainfall (mm)   73945 non-null  float64
 6   Highest 60 Min Rainfall (mm)   73851 non-null  float64
 7   Highest 120 Min Rainfall (mm)  73851 non-null  float64
 8   Mean Temperature (C)           24708 non-null  float64
 9   Maximum Temperature (C)        27208 non-null  float64
 10  Minimum Temperature (C)        27185 non-null  float64
 11  Mean Wind Speed (km/h)         28331 non-null  float64
 12  Max Wind Speed (km/h)          28663 non-null 

In [ ]:
# I pick out the columns that I want to use as features
feature_columns = df.columns[4:-3]
print(feature_columns)

Index(['Daily Rainfall Total (mm)', 'Highest 30 Min Rainfall (mm)',
       'Highest 60 Min Rainfall (mm)', 'Highest 120 Min Rainfall (mm)',
       'Mean Temperature (C)', 'Maximum Temperature (C)'],
      dtype='object')


In [ ]:
"""
aggregate_dfs = {}
# Generate Aggregate data (Monthly)
  print(i)
  aggregate_dfs[i] = df[['Year','Month',i]].dropna()
  aggregate_dfs[i] = aggregate_dfs[i].groupby(['Year','Month'])[i].mean()
  print(aggregate_dfs)
"""

# Generate Aggregate data (Weekly)
# First, merge the Year, Month & Day columns
# https://stackoverflow.com/questions/52358779/combining-year-month-day-columns-into-a-single-date-column-with-pandas
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
# Drop the columns we don't need
df = df.drop(columns=['Station', 'Year', 'Month', 'Day', 'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83715 entries, 0 to 83714
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Daily Rainfall Total (mm)      78874 non-null  float64       
 1   Highest 30 Min Rainfall (mm)   73945 non-null  float64       
 2   Highest 60 Min Rainfall (mm)   73851 non-null  float64       
 3   Highest 120 Min Rainfall (mm)  73851 non-null  float64       
 4   Mean Temperature (C)           24708 non-null  float64       
 5   Maximum Temperature (C)        27208 non-null  float64       
 6   Minimum Temperature (C)        27185 non-null  float64       
 7   Date                           83321 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(7)
memory usage: 5.1 MB
None


In [ ]:
# Inclusive check
df2 = df[(df['Date'] >= '31/12/2017') & (df['Date'] <= '02/01/2021')]
aggregate_dfs = {}
for feature in feature_columns:
  # https://stackoverflow.com/questions/38164453/dividing-pandas-dataframe-by-week
  print(feature)
  aggregate_dfs[feature] = df2[['Date',feature]].dropna()
  # Because the date should start at 31/12/2017, we have
  # aggregate_dfs[feature] = aggregate_dfs[feature].groupby(pd.Grouper(freq='W', key='Date'))[feature].agg(['count','min','mean','max'])
  aggregate_dfs[feature] = aggregate_dfs[feature].groupby(pd.Grouper(freq='W', key='Date'))[feature].agg(['mean','std'])
  aggregate_dfs[feature].to_csv(feature + '.csv')
  print(aggregate_dfs[feature])

Daily Rainfall Total (mm)
                 mean        std
Date                            
2017-12-31  68.556000  27.985034
2018-01-07  11.343658  18.038835
2018-01-14  21.103754  20.893830
2018-01-21   5.088398  10.804566
2018-01-28  10.938360  17.588305
...               ...        ...
2021-01-10  20.143910  37.470840
2021-01-17   2.775380   6.106402
2021-01-24   7.006553  11.702522
2021-01-31   2.509510   5.772267
2021-02-07   0.000000   0.000000

[163 rows x 2 columns]
Highest 30 Min Rainfall (mm)
                 mean       std
Date                           
2017-12-31  18.219565  7.831436
2018-01-07   5.363666  7.423016
2018-01-14   7.107547  9.960096
2018-01-21   3.711377  7.630486
2018-01-28   5.859714  8.436933
...               ...       ...
2021-01-10   4.563072  6.383830
2021-01-17   1.275617  3.074381
2021-01-24   5.016000  8.320522
2021-01-31   1.760641  3.934886
2021-02-07   0.000000  0.000000

[163 rows x 2 columns]
Highest 60 Min Rainfall (mm)
                 mean  